In [306]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpAffineExpression, \
LpConstraint, PULP_CBC_CMD
import numpy as np
import pulp as plp

In [353]:
foods = pd.read_csv('./Data_formatted/nv_df.csv', index_col='Unnamed: 0')
foods.index = [food.replace('-','_') for food in foods.index]
constraints = pd.read_csv('./Data_formatted/constraints.csv', index_col = 'Unnamed: 0')
constraints = constraints[1:] # remove calories
# Adjust amino acid requirements for bodyweight
usr_mass = input("How many kilos do you weigh?")
usr_mass = int(usr_mass)
constraints.iloc[-11:,0] *= usr_mass

How many kilos do you weigh? 75


In [385]:
def formulateMP(foods, constraints):
    '''This function will take two dataframes, one foods*nutrients, the
    other constraints*bounds and return the PuLP lp formulation for minimizing
    calories. '''
    
    # Decision Variables
    food_vars  = {food:
    plp.LpVariable(cat=plp.LpContinuous, 
                   lowBound=0,
                   name=f"{food}") 
    for food in foods.index.values}

    # Objective Function
    min_cals = plp.LpProblem("Min_Calories",plp.LpMinimize)
    min_cals += lpSum([foods.calories[food]*food_vars[food] 
                        for food in foods.index.values])
    # Add constraints
    for c in constraints.index:
        if not np.isnan(constraints.loc[c][0]):
            min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                              for food in foods.index]),
                                 sense=plp.LpConstraintGE, 
                                 rhs=constraints.loc[c][0],
                                 name=f'min_{c}'
                                )
        if not np.isnan(constraints.loc[c][1]):
            min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                              for food in foods.index]),
                                 sense=plp.LpConstraintLE, 
                                 rhs=constraints.loc[c][1],
                                 name=f'max_{c}'
                                )
    
    return min_cals

In [386]:
lp_formulation = formulateMP(foods,constraints)

In [356]:
lp_formulation.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/09fb77460b9f46edb62019bd1183696e-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/09fb77460b9f46edb62019bd1183696e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 65 COLUMNS
At line 5949 RHS
At line 6010 BOUNDS
At line 6011 ENDATA
Problem MODEL has 60 rows, 117 columns and 5766 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 181412.62 (38)
11  Obj 34942685 Primal inf 18101089 (24)
Primal infeasible - objective value 34942685
PrimalInfeasible objective 34942685.44 - 11 iterations 

-1

In [371]:
plp.listSolvers(onlyAvailable=True)

['PULP_CBC_CMD']

In [387]:
def checkFeasibility(foods, constraints):
    '''Formulate lp problem and return status and optimal coefficients if 
    feasible as 2-tuple'''
    
    min_cals = formulateMP(foods,constraints)
    min_cals.solve(PULP_CBC_CMD(msg=0))
    return min_cals.status, min_cals.coefficients

In [388]:
def infeasibilitySearch(foods, constraints):
    '''return 2d array with indices of constraints that constitute a solvable model
    and second the indices which cause an insolvable model when included.'''
    
    solvable = []
    brk_pts = []
    for i in range(0,constraints.shape[0]):
        candidates = [i for i in range(0,i+1) if i not in brk_pts]
        status = checkFeasibility(foods,constraints.iloc[candidates,:])[0]
        if (status == -1):
            brk_pts.append(i)
        else:
            solvable.append(i)
    return [solvable,brk_pts]

In [389]:
infeasibilitySearch(foods,constraints)

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28],
 [18, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]]

In [390]:
constraints.iloc[[18, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]]

,min,max
copper,900.0,10000.0
chloride,172.5,3.6
biotin,2250.0,NaN
chromium,2625.0,NaN
cysteine_methionine,1425.0,NaN
histidine,1050.0,NaN
leucine,3150.0,NaN
lysine,2850.0,NaN
methionine,975.0,NaN
phenylalanine_tyrosine,2475.0,NaN


Constraints with indices [18, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39] are not solvable when added to the formulation.  These constraints are: \
'copper', 'chloride', 'biotin', 'chromium', 'cysteine_methionine',
       'histidine', 'leucine', 'lysine', 'methionine',
       'phenylalanine_tyrosine', 'threonine', 'valine' \
\
It is interesting that many of these are amino acids.  



###### What do the loadings look like for the solution with the infeasible constraints removed?

In [398]:
i_c = [18, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
feasible = [i for i in range(constraints.shape[0]) if i not in i_c]
f_c = constraints.iloc[feasible]

In [392]:
model = formulateMP(foods,f_c)

In [393]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/cbf063bd95e741fd9c18e647b1b7a88f-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/cbf063bd95e741fd9c18e647b1b7a88f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 51 COLUMNS
At line 4752 RHS
At line 4799 BOUNDS
At line 4800 ENDATA
Problem MODEL has 46 rows, 117 columns and 4583 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 44 (-2) rows, 117 (0) columns and 4456 (-127) elements
0  Obj 0 Primal inf 1247.1462 (26)
31  Obj 1008.654
Optimal - objective value 1008.654
After Postsolve, objective 1008.654, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1008.654008 - 31 iterations time 0.002, Presolve 0.00

1

In [394]:
model.coefficients()

[('asparagus_cooked', 'min_protein', 4.32),
 ('avocado_cubed_raw', 'min_protein', 3.0),
 ('beet_greens_boiled', 'min_protein', 3.7),
 ('beets_sliced_cooked', 'min_protein', 2.86),
 ('bell_peppers_sliced_red_raw', 'min_protein', 0.91),
 ('bok_choy_cooked', 'min_protein', 2.65),
 ('broccoli_chopped_cooked', 'min_protein', 3.71),
 ('brussels_sprouts_cooked', 'min_protein', 3.98),
 ('cabbage_red_cooked', 'min_protein', 2.27),
 ('carrots_sliced_raw', 'min_protein', 1.13),
 ('cauliflower_cooked', 'min_protein', 2.28),
 ('celery_diced_raw', 'min_protein', 0.7),
 ('collard_greens_chopped_cooked', 'min_protein', 5.15),
 ('corn_yellow_cooked', 'min_protein', 2.63),
 ('cucumber_sliced_raw', 'min_protein', 0.68),
 ('eggplant_cubed_cooked', 'min_protein', 0.82),
 ('fennel_sliced_raw', 'min_protein', 1.08),
 ('garlic_raw', 'min_protein', 1.14),
 ('green_beans_cooked', 'min_protein', 2.36),
 ('green_peas_cooked', 'min_protein', 7.38),
 ('kale_cooked', 'min_protein', 2.47),
 ('leeks_cooked', 'min_prot

In [397]:
total_cals = 0
food_servings = {}
for v in model.variables():
    total_cals += v.varValue*foods.at[v.name,'calories']
    food_servings[v.name] = v.varValue

In [ ]:
constraintsp[]

In [ ]:
food_servings[v.name] = v.varValue

{'almonds_sliced_raw': 0.0,
 'apple_fresh': 0.0,
 'apricot_fresh': 0.0,
 'asparagus_cooked': 1.0899476,
 'avocado_cubed_raw': 0.0,
 'banana_fresh_af': 0.0,
 'barley_hulled_dry': 0.0,
 'basil_chopped_fresh': 1.9565506,
 'beef_grass_fed_strip_steak_cooked': 0.0,
 'beet_greens_boiled': 0.0,
 'beets_sliced_cooked': 0.0,
 'bell_peppers_sliced_red_raw': 0.0,
 'black_beans_cooked': 0.0,
 'black_pepper_whole_peppercorns': 0.0,
 'blueberries_fresh': 0.0,
 'bok_choy_cooked': 0.0,
 'broccoli_chopped_cooked': 0.0,
 'brown_rice_long_grain_cooked': 0.0,
 'brussels_sprouts_cooked': 0.0,
 'buckwheat_groats_cooked': 0.0,
 'cabbage_red_cooked': 0.0,
 'cantaloupe_cubed_fresh': 0.0,
 'carrots_sliced_raw': 0.0,
 'cashews_raw': 0.0,
 'cauliflower_cooked': 2.5395574,
 'celery_diced_raw': 0.0,
 'cheese_grass_fed_cheddar_whole_milk': 0.0,
 'chicken_pasture_raised_breast_roasted': 0.0,
 'chili_peppers_hot_dried': 14.246776,
 'cilantro_leaf_fresh': 0.0,
 'cinnamon_ground': 0.0,
 'cloves_ground': 2.3867693,
 'cod

In [396]:
total_cals

1008.65401727822

In [351]:
sorted(foods.index.values)

['almonds_sliced_raw',
 'apple_fresh',
 'apricot_fresh',
 'asparagus_cooked',
 'avocado_cubed_raw',
 'banana_fresh_af',
 'barley_hulled_dry',
 'basil_chopped_fresh',
 'beef_grass-fed_strip_steak_cooked',
 'beet_greens_boiled',
 'beets_sliced_cooked',
 'bell_peppers_sliced_red_raw',
 'black_beans_cooked',
 'black_pepper_whole_peppercorns',
 'blueberries_fresh',
 'bok_choy_cooked',
 'broccoli_chopped_cooked',
 'brown_rice_long_grain_cooked',
 'brussels_sprouts_cooked',
 'buckwheat_groats_cooked',
 'cabbage_red_cooked',
 'cantaloupe_cubed_fresh',
 'carrots_sliced_raw',
 'cashews_raw',
 'cauliflower_cooked',
 'celery_diced_raw',
 'cheese_grass_fed_cheddar_whole_milk',
 'chicken_pasture-raised_breast_roasted',
 'chili_peppers_hot_dried',
 'cilantro_leaf_fresh',
 'cinnamon_ground',
 'cloves_ground',
 'cod_pacific_fillet_baked',
 'collard_greens_chopped_cooked',
 'corn_yellow_cooked',
 "cow's_milk_grass-fed",
 'cranberries_fresh_whole',
 'cucumber_sliced_raw',
 'cumin_seeds',
 'dill_sprig_fre

In [335]:
servings = [food[2] for food in model.coefficients()][0:foods.shape[0]]

In [340]:
# 109,835 calories under constraints -- this result requires investigation.  
sum(servings*foods.calories)

109834.88880000004

117

In [260]:
[c for c in range(0,40) if c not in candidates ]

[18, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]

In [262]:
constraints.index.values[[c for c in range(0,40) if c not in candidates ]]

array(['copper', 'chloride', 'biotin', 'chromium', 'cysteine_methionine',
       'histidine', 'leucine', 'lysine', 'methionine',
       'phenylalanine_tyrosine', 'threonine'], dtype=object)

In [263]:
# narrowing the constraints to non-macros, which may have been setting a floor on
# the total number of calories that limits optimization

macros = ['protein', 'fat_total', 'saturated_fat', 'carbohydrates']
constraints_micros_aa = constraints.loc[[c for c in constraints.index.values if 
                                         c not in macros]]

'copper', 'chloride', 'biotin', 'chromium', 'cysteine_methionine', 'histidine', 'leucine', 'lysine', 'methionine', 'phenylalanine_tyrosine', and 'threonine' are not solvable when all constraints are used in search.

In [198]:
vitamins = ['choline', 'folate', 'niacin', 'pantothenic_acid', 'vitamin_b2',
       'vitamin_b1', 'vitamin_a', 'vitamin_b12', 'vitamin_b6', 'vitamin_c',
       'vitamin_d_mcg', 'vitamin_k', 'calcium', 'copper', 'iron', 'magnesium',
       'manganese', 'phosphorus', 'potassium', 'selenium', 'zinc']

for v in vitamins[0:5]:
    if not np.isnan(constraints.loc[v][0]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,v]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintGE, 
                             rhs=constraints.loc[v][0],
                             name=f'min_{v}'
                            )
    if not np.isnan(constraints.loc[v][1]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,v]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintLE, 
                             rhs=constraints.loc[v][1],
                             name=f'max_{v}'
                            )

In [200]:
min_cals.solve()
min_cals.objective.value()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/0e6c6d9e90ee4ed390faa177aa9fe682-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/0e6c6d9e90ee4ed390faa177aa9fe682-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 803 RHS
At line 810 BOUNDS
At line 811 ENDATA
Problem MODEL has 6 rows, 117 columns and 674 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 6 (0) rows, 117 (0) columns and 674 (0) elements
0  Obj 0 Primal inf 5.2153918 (3)
4  Obj 325.23604
Optimal - objective value 325.23604
Optimal objective 325.2360389 - 4 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock se

325.236035646

In [120]:
for c in constraints.index[]:
    if not np.isnan(constraints.loc[c][0]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintGE, 
                             rhs=constraints.loc[c][0],
                             name=f'min_{c}'
                            )
    if not np.isnan(constraints.loc[c][1]):
        min_cals += LpConstraint(e=lpSum([foods.loc[food,c]*food_vars[food] 
                                          for food in foods.index]),
                             sense=plp.LpConstraintLE, 
                             rhs=constraints.loc[c][1],
                             name=f'max_{c}'
                            )

In [ ]:
core_constraints = ['protein', 'cysteine_methionine', 'histidine', 'leucine', 'lysine',
       'methionine', 'phenylalanine_tyrosine', 'threonine', 'valine']

In [122]:
constraints.index

Index(['protein', 'fat_total', 'saturated_fat', 'cholesterol', 'sodium',
       'choline', 'folate', 'niacin', 'pantothenic_acid', 'vitamin_b2',
       'vitamin_b1', 'vitamin_a', 'vitamin_b12', 'vitamin_b6', 'vitamin_c',
       'vitamin_d_mcg', 'vitamin_k', 'calcium', 'copper', 'iron', 'magnesium',
       'manganese', 'phosphorus', 'potassium', 'selenium', 'zinc',
       'carbohydrates', 'fiber', 'molybdenum', 'chloride', 'biotin',
       'chromium', 'cysteine_methionine', 'histidine', 'leucine', 'lysine',
       'methionine', 'phenylalanine_tyrosine', 'threonine', 'valine'],
      dtype='object')

In [121]:
min_cals.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/williammohr/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/9346b6b162f84f999f60d9faf9ba9323-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/9346b6b162f84f999f60d9faf9ba9323-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 65 COLUMNS
At line 5949 RHS
At line 6010 BOUNDS
At line 6011 ENDATA
Problem MODEL has 60 rows, 117 columns and 5766 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 183679.88 (38)
11  Obj 35420182 Primal inf 18349995 (24)
Primal infeasible - objective value 35420182
PrimalInfeasible objective 35420182.37 - 11 iterations 

-1

In [ ]:
dir(plp);

In [ ]:
# Constraints

plp.Lp

In [ ]:
pulp.lpSum

In [ ]:
protein_calories = foods['protein']/foods['calories'].sort_values(ascending=False).copy()

In [ ]:
protein_calories.sort_values(ascending=False)

In [ ]:
# selecting high protein to calorie ratio foods
protein_foods = foods.loc[protein_calories.sort_values(ascending=False)[0:40].index,:]\
.query("calories > 50")

In [ ]:
constraints.drop(['saturated_fat','cholesterol'])

In [ ]:
necessary_nutrients = [c for c in constraints.index if c not in ['saturated_fat','cholesterol']]

In [ ]:
constraints.loc[~constraints.index.isin(['saturated_fat','cholesterol']),:].index.values

In [ ]:
foods[necessary_nutrients].idxmax()

In [ ]:
max_foods = foods.loc[list(set(foods[necessary_nutrients].idxmax().values)),:]

In [ ]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in max_foods.index],
}

data_foods.update({col: protein_foods[col] for col in protein_foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Integer')

# Define the objective function to minimize the total number of servings
problem += lpSum(food_vars[food] for food in df_foods['Food'])

# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Next I will solve to minimize calories since minimizing for serving size has no solution.

In [ ]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
problem += lpSum(food_vars[food] for food in df_foods['Food'])

# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

Looking to minimize calories:

In [ ]:

# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
ae_calories = LpAffineExpression([(food_vars[food],df_foods.loc[food,'calories']) for food in df_foods['Food']])
problem += ae_calories
# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")

In [ ]:
constraints.drop(index=['calories'],inplace=True)
# Sample data for foods and nutrients
data_foods = {
    'Food': [food for food in foods.index],
}

data_foods.update({col: foods[col] for col in foods.columns})

df_foods = pd.DataFrame(data_foods)

# Sample data for constraints
data_constraints = {
    'Nutrient': [nutrient for nutrient in constraints.index],
    'MinValue': [val for val in constraints['min']],
    'MaxValue': [val for val in constraints['max']],
}

df_constraints = pd.DataFrame(data_constraints)

# Create the LP Minimization problem
problem = LpProblem("Nutritionally_Complete_Foods", LpMinimize)

# Define decision variables
food_vars = LpVariable.dicts("Servings", df_foods['Food'], lowBound=0, cat='Continuous')

# Define the objective function to minimize the total number of servings
ae_calories = LpAffineExpression([(food_vars[food],df_foods.loc[food,'calories']) for food in df_foods['Food']])
problem += ae_calories
# Add constraints based on nutrient requirements
for nutrient in df_constraints['Nutrient']:
    min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MinValue'].values[0]
    max_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 'MaxValue'].values[0]
    
    if not pd.isna(min_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    >= min_value
    if not pd.isna(max_value):
        problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
                    food, nutrient].values[0] for food in df_foods['Food']) \
                    <= max_value

# # Add constraint for nutritional completeness
# for nutrient in df_constraints['Nutrient']:
#     min_value = df_constraints.loc[df_constraints['Nutrient'] == nutrient, 
#                             'MinValue'].values[0]
#     problem += lpSum(food_vars[food] * df_foods.loc[df_foods['Food'] == 
#                 food, nutrient].values[0] for food in df_foods['Food']) 
#                 >= min_value

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)

if problem.status == 1:  # If the problem was solved successfully
    for food in df_foods['Food']:
        servings = food_vars[food].value()
        if servings > 0:
            print(f"{food}: {servings} servings")
else:
    print("No solution found.")